<a href="https://colab.research.google.com/github/zooodung/Face_Image_Emotion_Classification/blob/JS/FP_ViTb16_Test_Data_Result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Project/ViT_b16')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: vit_keras.vit is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/lambda_layer.py:327: UserWarning: vit_keras.layers is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [ ]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from skimage import color
from tensorflow.keras.preprocessing import image
import dlib
from google.colab.patches import cv2_imshow
from mtcnn import MTCNN

In [ ]:
import json

with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_anger.json', "r", encoding='euc-kr') as f:
    anger_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_happy.json', "r", encoding='euc-kr') as f:
    happy_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_panic.json', "r", encoding='euc-kr') as f:
    panic_label = json.load(f)
with open ('/content/drive/MyDrive/Project/Data/Test_data/label/test_sadness.json', "r", encoding='euc-kr') as f:
    sadness_label = json.load(f)

In [ ]:
import zipfile
import os

def unzip_file(zip_file_path, extract_dir):
    # 압축 파일 열기
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 압축 해제
        zip_ref.extractall(extract_dir)

# 압축 파일 경로와 추출할 디렉토리 경로 지정
zip_file_path = '/content/drive/MyDrive/Project/Data/test_img.Zip'
extract_dir = '/content/test'

# 압축 파일 풀기
unzip_file(zip_file_path, extract_dir)

In [ ]:
import tensorflow as tf
import os
import logging
from mtcnn import MTCNN

def extract_face_info_mtcnn(img):

  detector = MTCNN() # MTCNN 인스턴스 생성

  faces = detector.detect_faces(img) # 얼굴 감지
  #face_info_list = [] # 감지된 얼굴 정보를 저장할 리스트 // 하나의 이미지에 다수의 얼굴이 존재할 경우 사용

  # 감지된 모든 얼굴에 대해 반복
  for face in faces:
    # 얼굴의 바운딩 박스와 특징점 추출
    bounding_box = face['box']
    keypoints = face['keypoints']

    # 추출한 정보를 딕셔너리로 저장
    face_info = {
      'bounding_box': bounding_box,
      'keypoints': keypoints
    }

    # 얼굴 정보 리스트에 추가
    #face_info_list.append(face_info)

  return face_info


def calculate_angle(face_info):
  left_eye = face_info['keypoints']['left_eye']
  right_eye = face_info['keypoints']['right_eye']

  return np.arctan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]) * 180 / np.pi

def rotate_image(image, img_info):
  angle = calculate_angle(img_info)
  # 이미지의 중심 탐색
  height, width = image.shape[:2]
  center = (width / 2, height / 2)

  # 회전 변환 매트릭스 생성
  rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

  # 이미지 회전
  rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

  return rotated_image

def normalize_face(image, face_info, output_size=(224, 224)):
    landmarks = face_info['keypoints']
    left_eye = np.array(landmarks['left_eye'])
    right_eye = np.array(landmarks['right_eye'])
    nose_tip = np.array(landmarks['nose'])

    # 눈 사이 거리
    eye_distance = np.linalg.norm(left_eye - right_eye)

    # 스케일링 비율
    desired_eye_distance = 0.3 * output_size[0]
    scale = desired_eye_distance / eye_distance

    # 눈 사이 거리 정규화
    M = np.array([[scale, 0, 0], [0, scale, 0]])
    scaled_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # 코 끝을 기준으로 중심 이동
    nose_center = nose_tip * scale
    offset_x = (output_size[0] / 2) - nose_center[0]
    offset_y = (output_size[1] / 2) - nose_center[1]
    M = np.array([[1, 0, offset_x], [0, 1, offset_y]])
    normalized_image = cv2.warpAffine(scaled_image, M, output_size)

    return normalized_image

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('/content/drive/MyDrive/Project/Data/shape_predictor_68_face_landmarks.dat')

def extract_face_info_dlib(img):
  faces = detector(img, 1)

  #if len(faces) == 0:
  #  print("얼굴을 찾을 수 없습니다.")
  #  return

  for face in faces:
    # 얼굴 영역 추출
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    face_area = img[y1:y2, x1:x2]

    # 얼굴 특징점 추출
    landmarks = predictor(img, face)

  return landmarks

In [ ]:
# 이미지 가져오기
def get_image(filename, path):
  image = cv2.imread(path + filename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image.copy()

# Label 데이터를 기반으로 이미지 Crop하는 함수
def crop_face(label_data, img):

  boxes = [label_data['annot_A']['boxes'], label_data['annot_B']['boxes'], label_data['annot_C']['boxes']]
  avg_box = { # annot_A/B/C 각 좌표의 평균 값 구해서 저장
      'maxX': np.mean([box['maxX'] for box in boxes]),
      'maxY': np.mean([box['maxY'] for box in boxes]),
      'minX': np.mean([box['minX'] for box in boxes]),
      'minY': np.mean([box['minY'] for box in boxes])
  }

  new_image = img[int(avg_box['minY']):int(avg_box['maxY']), int(avg_box['minX']):int(avg_box['maxX'])] # 평균값으로 이미지 crop

  return new_image

In [ ]:
# 감정 레이블 정의
emotion_labels = ['anger', 'happy', 'panic', 'sadness']

# 이미지 전처리
def preprocess_image(label, path):
  img = get_image(label['filename'], path) # 이미지 불러오기
  img = crop_face(label, img) # 이미지 크롭
  img_info = extract_face_info_mtcnn(img) # 이미지에서 얼굴 정보 추출
  img = rotate_image(img, img_info) # 눈 수평 회전
  img = normalize_face(img, img_info) # 코 기준 이미지 센터로 이동
  img_array = np.expand_dims(img, axis=-1)  # 마지막 차원에 채널 추가
  img_array = img_array.astype('float32') / 255.0  # 0~1 사이 값으로 정규화
  img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

  return img_array

# 감정 분류 함수
def predict_emotion(img_array):
  predictions = model.predict(img_array)
  predicted_class = np.argmax(predictions)
  predicted_emotion = emotion_labels[predicted_class]
  confidence = predictions[0][predicted_class]

  return predicted_emotion, confidence

# 이미지 전처리 및 감정 예측
def get_predict_accuracy(label, path, true_emotion):
  correct_count = 0
  error_count = 0
  confidence_count = 0

  for i in range(len(label)):
    try:
      img_array = preprocess_image(label[i], path)
      predicted_emotion, confidence = predict_emotion(img_array)
      #true_emotion = label[i]['faceExp_uploader']
    except Exception as e:
      error_count += 1
      continue

    if predicted_emotion == true_emotion:
      correct_count += 1
      confidence_count += confidence

    filename = label[i]['filename']
    print(f'{i+1}, filename : {filename}')
    print(f'예측 : {predicted_emotion}\n실제 : {true_emotion}\n신뢰도 : {confidence:.2f}')

  return correct_count, error_count, confidence_count

In [ ]:
anger_correct_count, anger_error_count, anger_confidence_count = get_predict_accuracy(anger_label, '/content/test/anger/', 'anger')
happy_correct_count, happy_error_count, happy_confidence_count = get_predict_accuracy(happy_label, '/content/test/happy/', 'happy')
panic_correct_count, panic_error_count, panic_confidence_count = get_predict_accuracy(panic_label, '/content/test/panic/', 'panic')
sadness_correct_count, sadness_error_count, sadness_confidence_count = get_predict_accuracy(sadness_label, '/content/test/sadness/', 'sadness')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 41ms/step
8, filename : xje3c2677a87f22fc3e161778238e79f20c0f325e47decf5717bf314c118dxvkf.jpg
예측 : sadness
실제 : sadness
신뢰도 : 0.89
1/1 [==============================] - 0s 42ms/step
9, filename : rur1a573d7212f12127f2ebde123a22f21882f0f1f22fe523c2fa3fa3e2bfmozo.jpg
예측 : sadness
실제 : sadness
신뢰도 : 1.00
1/1 [==============================] - 0s 47ms/step
10, filename : svbp8d38f3d8cc0298d648fd83192c1d19e5d47ec3170229811dae5d701f7hhqg.jpg
예측 : panic
실제 : sadness
신뢰도 : 0.99
1/1 [==============================] - 0s 48ms/step
11, filename : tzz0c62294472ea7485ae919e172e27d9e542499189969cc8517dd0de72745vvk.jpg
예측 : sadness
실제 : sadness
신뢰도 : 0.89
1/1 [==============================] - 0s 48ms/step
12, filename : c5av127b3bb3c8c88d65dccc88a8b66f17888bfa11e5e762d11eb98e1951b98ec.jpg
예측 : sadness
실제 : sadness
신뢰도 : 0.99
1/1 [==============================] - 0s 46ms/step
13, filename : s14i4268fb0c02c3fc3bc8098aff482

In [ ]:
entire_correct_count = anger_correct_count + happy_correct_count + panic_correct_count + sadness_correct_count
entire_error_count = anger_error_count + happy_error_count + panic_error_count + sadness_error_count
entire_lable_length = len(anger_label) + len(happy_label) + len(panic_label) + len(sadness_label)
entire_emotion_accuracy = entire_correct_count / (entire_lable_length - entire_error_count) * 100

In [ ]:
# 예측을 위해 테스트 데이터 전처리를 시도했지만 전처리에 실패한 개수
print(f'Anger Error Image Count : {anger_error_count}')
print(f'Happy Error Image Count : {happy_error_count}')
print(f'Panic Error Image Count : {panic_error_count}')
print(f'Sadness Error Image Count : {sadness_error_count}')

Anger Error Image Count : 4
Happy Error Image Count : 8
Panic Error Image Count : 5
Sadness Error Image Count : 4


In [ ]:
# 예측 감정과 정답 감정이 일치한 개수
print(f'Anger Correct Count : {anger_correct_count}')
print(f'Happy Correct Count : {happy_correct_count}')
print(f'Panic Correct Count : {panic_correct_count}')
print(f'Sadness Correct Count : {sadness_correct_count}')

Anger Correct Count : 194
Happy Correct Count : 280
Panic Correct Count : 239
Sadness Correct Count : 253


In [ ]:
# 예측 감정과 정답 감정이 일치할 때 신뢰도의 평균
print(f'Mean of Anger confidence : {anger_confidence_count/anger_correct_count:.2f}')
print(f'Mean of Happy confidence : {happy_confidence_count/happy_correct_count:.2f}')
print(f'Mean of Panic confidence : {panic_confidence_count/panic_correct_count:.2f}')
print(f'Mean of Sadness confidence : {sadness_confidence_count/sadness_correct_count:.2f}')

Mean of Anger confidence : 0.93
Mean of Happy confidence : 0.99
Mean of Panic confidence : 0.97
Mean of Sadness confidence : 0.96


In [ ]:
# 감정별 예측 정확도
print(f'Anger Predict Accuracy : {anger_correct_count / (len(anger_label) - anger_error_count) * 100:.2f}%')
print(f'Happy Predict Accuracy : {happy_correct_count / (len(happy_label) - happy_error_count) * 100:.2f}%')
print(f'Panic Predict Accuracy : {panic_correct_count / (len(panic_label) - panic_error_count) * 100:.2f}%')
print(f'Sadness Predict Accuracy : {sadness_correct_count / (len(sadness_label) - sadness_error_count) * 100:.2f}%')
# 전체 테스트 데이터셋에 대한 감정 예측 정확도
print(f'\nEntire Emotion Accuracy : {entire_emotion_accuracy:.2f}%')

Anger Predict Accuracy : 70.29%
Happy Predict Accuracy : 96.55%
Panic Predict Accuracy : 88.52%
Sadness Predict Accuracy : 90.36%

Entire Emotion Accuracy : 84.59%


In [ ]:
def print_faceExp_count(counts, emotion):
    print(f"Counts for emotion '{emotion}'")
    print(f"  Zero annotation same   : {counts['zero']}")
    print(f"  One annotation same    : {counts['one']}")
    print(f"  Two annotations same   : {counts['two']}")
    print(f"  Three annotations same : {counts['three']}")
    print(f"  Sum of two, three : {counts['two'] + counts['three']}")
    print(f"  Sum of all        : {counts['zero'] + counts['one'] + counts['two'] + counts['three']}")

def verify_faceExp_consistent(label, emotion):
  counts = {'zero': 0, 'one': 0, 'two': 0, 'three': 0}
  verified_data = []

  for i in range(len(label)):
    emotion_count = 0
    for annot in ['annot_A', 'annot_B', 'annot_C']:
      if label[i][annot]['faceExp'] == emotion:
        emotion_count += 1

    if emotion_count == 1:
      counts['one'] += 1
    elif emotion_count == 2: # 2개 이상 일치시 리스트 추가
      counts['two'] += 1
      verified_data.append(label[i])
    elif emotion_count == 3: # 3개 일치시 리스트 추가
      counts['three'] += 1
      verified_data.append(label[i])
    else :
      counts['zero'] += 1

  print_faceExp_count(counts, emotion)

  return verified_data

In [ ]:
verified_anger = verify_faceExp_consistent(anger_label, '분노')
verified_happy = verify_faceExp_consistent(happy_label, '기쁨')
verified_panic = verify_faceExp_consistent(panic_label, '당황')
verified_sadness = verify_faceExp_consistent(sadness_label, '슬픔')

Counts for emotion '분노'
  Zero annotation same   : 28
  One annotation same    : 47
  Two annotations same   : 71
  Three annotations same : 134
  Sum of two, three : 205
  Sum of all        : 280
Counts for emotion '기쁨'
  Zero annotation same   : 2
  One annotation same    : 5
  Two annotations same   : 12
  Three annotations same : 279
  Sum of two, three : 291
  Sum of all        : 298
Counts for emotion '당황'
  Zero annotation same   : 18
  One annotation same    : 29
  Two annotations same   : 79
  Three annotations same : 149
  Sum of two, three : 228
  Sum of all        : 275
Counts for emotion '슬픔'
  Zero annotation same   : 29
  One annotation same    : 43
  Two annotations same   : 75
  Three annotations same : 137
  Sum of two, three : 212
  Sum of all        : 284
